<a href="https://colab.research.google.com/github/Somya2119/Capston-Project-Bike-Sharing-Count-Prediction/blob/main/_capstone_project_collab_group_notebook_bike_sharing_count_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Bike Sharing Price Prediction



##### **Project Type**    - Regression
##### **individual -** - Somya Jain



# **Project Summary -**

 First we do Exploratory Data Analysis on the data set. We look for missing data values,duplicates values (none were found) and outliers and appropriately modify them. we have seen relation b/w numeric features and dependent variables and seen seasonwise and weekdays- weekend monthly distribution of rented bike count. We also perform correlation analysis to extract out the important and relevant feature set and later perform feature engineering to modify few existing columns and drop out irrelavant ones.we have seen skewness in data set  and to remove skewness we have used square root transforming.

We then look at several popular individual models from simple ones like Linear Regressor and Regularization Models (Ridge and Lasso) to more complicated ensemble ones like Decision Tree, Random Forest, XG Boost and Light GBM. 


# **GitHub Link -**

Provide your GitHub Link here.  https://github.com/Somya2119/Capston-Project-Bike-Sharing-Count-Prediction.git

# **Problem Statement**


**Currently Rental bikes are introduced in many urban cities for the enhancement of mobility comfort. It is important to make the rental bike available and accessible to the public at the right time as it lessens the waiting time. Eventually, providing the city with a stable supply of rental bikes becomes a major concern. The crucial part is the prediction of bike count required at each hour for the stable supply of rental bikes.**

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
from numpy import math
from numpy import loadtxt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

from scipy.stats import *
import math

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from xgboost import XGBRFClassifier
from sklearn.tree import export_graphviz

!pip install shap==0.40.0
import shap 
import graphviz
sns.set_style('darkgrid') 

import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.6/567.6 KB 11.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing the dataset
rental_bike_df = pd.read_csv('/content/drive/MyDrive/Capstone project-2 Machine Learning(Regression)/SeoulBikeData .csv',encoding='ISO-8859-1')

### Dataset First View

In [ ]:
# Dataset First Look
rental_bike_df.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [ ]:
rental_bike_df.shape

In [ ]:
rental_bike_df.describe()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
rental_bike_df.shape

### Dataset Information

In [ ]:
# Dataset Info
rental_bike_df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
len(rental_bike_df[rental_bike_df.duplicated()])

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
rental_bike_df.isna().sum()

In [ ]:
# Visualizing the missing values
plt.figure(figsize=(14, 5))
sns.heatmap(rental_bike_df.isnull(), cbar=True, yticklabels=False,cmap="Blues")
plt.xlabel("column_name", size=14, weight="bold")
plt.title("missing values in column",fontweight="bold",size=17)
plt.show()

## What did you know about your dataset?

### The dataset contains weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour and date information.
### Attribute Information:
* ### **Date** : year-month-day
* ### **Rented Bike count:** Count of bikes rented at each hour
* ### **Hour** - Hour of the day
* ### **Temperature:**Temperature in Celsius
* ### **Humidity:**-%
* ### **Windspeed :**-m/s
* ### **Visibility:**-10m
* ### **Dew point temperature:**-Celsius
* ### **Solar radiation :**-MJ/m2
* ### **Rainfall:**-mm
* ### **Snowfall :**-cm
* ### **Seasons :**- Winter, Spring, Summer, Autumn
* ### **Holiday :**- Holiday/No holiday
* ### **Functional Day :**-- NoFunc(Non Functional Hours), Fun(Functional hours)

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
rental_bike_df.columns

In [ ]:
# Dataset Describe
rental_bike_df.describe()

### Variables Description 

we get to know about our data is:

*   Four features are of integer data types.
*   Six features are of float data types.
*   Four features are of object data types.
*   Also there are no null values.

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
for i in ['Seasons','Holiday','Functioning Day']:
  print(rental_bike_df[i].value_counts())

## 3. ***Data Wrangling***

**Exploratory Data Analysis**

In [ ]:
# Numeric Features

numeric_features = rental_bike_df.describe().columns
numeric_features

In [ ]:
# creating a dataframe containing the sum of all column values pertaining to differant seasons

df_1 = rental_bike_df.groupby('Seasons').sum()
df_1

In [ ]:
# creating a pie chart of bike count in differant seasons

df_1['Rented Bike Count'].plot(kind='pie', subplots=True, figsize=(8, 8), autopct = "%1.1f%%")


conclusions from above pie chart:

 Most bikes have been rented in the summer season.
least bike rent count is in winter season.
autumn and spring seasons have almost equal amounts of bike rent count.


In [ ]:
# Working on Categorical features
# Date columns to Date format conversion
rental_bike_df['Date']= pd.to_datetime(rental_bike_df['Date'])


In [ ]:
# creating a column containing the year from a particular date

year = []
for i in range(len(rental_bike_df['Date'])):
  year.append(rental_bike_df['Date'][i].year)
rental_bike_df['year'] = year  


In [ ]:
# creating a series which shows total number of bikes rented in each year

df_year = rental_bike_df.groupby('year').sum()['Rented Bike Count']
df_year

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
df_year.plot(kind='bar', ax=ax)




Above plot shows that most of the bikes have been rented in the year 2018.



In [ ]:
# creating a series which shows total number of bikes rented on the type of day

df_hol = rental_bike_df.groupby('Holiday').sum()
df_hol

In [ ]:
df_hol['Rented Bike Count'].plot(kind='pie', subplots=True, figsize=(5, 5) ,autopct = "%1.1f%%" )

Above plot shows that most of the bikes have been rented on working days.



In [ ]:
# bikes rented on type of day in each year

plt.figure(figsize=(8,7))
sns.countplot(x='Holiday', hue = 'year', data= rental_bike_df)
plt.title('Density of rented bikes on holiday vs no holiday in a particular year')
plt.show()


In [ ]:
# extracting day,month, day of week and weekdays/weekend from date column
rental_bike_df['month'] = rental_bike_df['Date'].apply(lambda x : x.month)
rental_bike_df['day_of_week'] = rental_bike_df['Date'].dt.day_name()
rental_bike_df['weekdays_weekend']=rental_bike_df['day_of_week'].apply(lambda x : 1 if x=='Saturday' or x=='Sunday' else 0 )
#rental_bike_df=rental_bike_df.drop(columns=['Date','day_of_week'],axis=1)


In [ ]:
rental_bike_df.head()

In [ ]:
# plot showing the bikes rented in differant months in the year 2018

sns.countplot(rental_bike_df.loc[rental_bike_df['year']==2018,'month'])


 Above plot shows that very less bikes have been rented in december which is winter season.



In [ ]:
# creating a dataframe containing the count of bikes rented in differant intensities of rainfall

df_rain = pd.DataFrame(rental_bike_df.groupby('Rainfall(mm)')['Rented Bike Count'].sum())


In [ ]:
# resetting index of the dataframe

df_rain.reset_index(inplace=True)
df_rain.head()


In [ ]:
# plot showing distribution of bike rentals according to rainfall intensity

sns.distplot(df_rain['Rainfall(mm)'])


Above plot shows that people tend to rent bikes when there is no or less rainfall.



In [ ]:
# creating a dataframe containing the count of bikes rented in differant intensities of snowfall

df_snow = pd.DataFrame(rental_bike_df.groupby('Snowfall (cm)')['Rented Bike Count'].sum())
df_snow.reset_index(inplace=True)


In [ ]:
df_snow.head()

In [ ]:
# plot showing distribution of bike rentals according to snowfall intensity

sns.distplot(df_snow['Snowfall (cm)'])

Above plot shows that people tend to rent bikes when there is no or less snowfall.



In [ ]:
# creating a dataframe containing the count of bikes rented in differant intensities of tempreature

df_temp = pd.DataFrame(rental_bike_df.groupby('Temperature(°C)')['Rented Bike Count'].sum())
df_temp.reset_index(inplace=True)
df_temp.head()


In [ ]:
# plot showing distribution of bike rentals according to temperature intensity

sns.distplot(df_temp['Temperature(°C)'])


Above plot shows that people tend to rent bikes when the temperature is between -5 to 25 degrees.



In [ ]:
# creating a dataframe containing the count of bikes rented in differant visibility ranges

df_visibility = pd.DataFrame(rental_bike_df.groupby('Visibility (10m)')['Rented Bike Count'].sum())
df_visibility.reset_index(inplace=True)
df_visibility.head()

In [ ]:
sns.distplot(df_visibility['Visibility (10m)'])


Above plot shows that people tend to rent bikes when the visibility is between 300 to 1700.


In [ ]:
# creating a dataframe containing the count of bikes rented in differant intensities of humidity

df_humidity = pd.DataFrame(rental_bike_df.groupby('Humidity(%)')['Rented Bike Count'].sum())
df_humidity.reset_index(inplace=True)
df_humidity.head()

In [ ]:
sns.distplot(df_humidity['Humidity(%)'])


In [ ]:
# scatter plot of bike count at hour of a particular day

plt.scatter(rental_bike_df['Hour'], rental_bike_df['Rented Bike Count'], alpha=0.5)
plt.title('Scatter plot of bike count with hour')
plt.xlabel('hour')
plt.ylabel('bike count')
plt.show()


 From above its clear that the rentals were more in the morning and evening.This is because people not having personal vehicle, commuting to offices and schools tend to rent bikes.



In [ ]:
f, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 18))
group_work_hour = pd.DataFrame(rental_bike_df.groupby(['Holiday', 'Hour'])['Rented Bike Count'].mean()).reset_index()
sns.pointplot(data=group_work_hour, x='Hour', y='Rented Bike Count', hue='Holiday', ax=axes[0], legend=True)
handles, _ = axes[0].get_legend_handles_labels()
axes[0].legend(handles, ['Not a Working Day', 'Working Day'])
axes[0].set(xlabel='Hour in the day', ylabel='Count', title='Average Bike Rentals by the day if Working day or Not')

hue_order= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
group_day_hour = pd.DataFrame(rental_bike_df.groupby(['day_of_week', 'Hour'])['Rented Bike Count'].mean()).reset_index()
sns.pointplot(data=group_day_hour, x='Hour', y='Rented Bike Count', hue='day_of_week', ax=axes[1], hue_order=hue_order)
axes[1].set(xlabel='Hour in the day', ylabel='Count', title='Average Bike Rentals by the day across Weekdays')



plt.show()



In [ ]:
## plot a bar plot for each categorical feature count (except Date & Rented Bike Count)
for col in numeric_features[1:-1]:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    feature = rental_bike_df[col]
    label = rental_bike_df['Rented Bike Count']
    correlation = feature.corr(label)
    plt.scatter(x=feature, y=label)
    plt.xlabel(col)
    plt.ylabel('Rented Bike Count')
    ax.set_title('Rented Bike Count vs ' + col + '- correlation: ' + str(correlation))
    z = np.polyfit(rental_bike_df[col], rental_bike_df['Rented Bike Count'], 1)
    y_hat = np.poly1d(z)(rental_bike_df[col])

    plt.plot(rental_bike_df[col], y_hat, "r--", lw=1)

plt.show()

In [ ]:
#checking categorical variable
categorical_features = rental_bike_df.describe(include=['object','category']).columns

In [ ]:
categorical_features

In [ ]:
# plot a bar plot for each categorical feature count  

for col in categorical_features:
    counts = rental_bike_df[col].value_counts().sort_index()
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    counts.plot.bar(ax = ax, color='red')
    ax.set_title(col + ' counts')
    ax.set_xlabel(col) 
    ax.set_ylabel("Frequency")
plt.show()

In [ ]:
#relation between categoricals and dependent features
fig,ax=plt.subplots(figsize=(15,8))
sns.set_style('white')
#Bar plot for seasonwise monthly distribution of counts
sns.barplot(x='month',y='Rented Bike Count',data=rental_bike_df[['month','Rented Bike Count','Seasons']],hue='Seasons',ax=ax)
ax.set_title('Seasonwise monthly distribution of counts')
plt.show()
#Bar plot for weekday wise monthly distribution of counts
fig,ax1=plt.subplots(figsize=(15,8))
sns.barplot(x='month',y='Rented Bike Count',data=rental_bike_df[['month','Rented Bike Count','weekdays_weekend']],hue='weekdays_weekend',ax=ax1)
ax.set_title('Weekdays wise monthly distribution of counts')
plt.show()
#Bar plot for weekday wise monthly distribution of counts
fig,ax1=plt.subplots(figsize=(15,8))
sns.barplot(x='Holiday',y='Rented Bike Count',data=rental_bike_df[['Holiday','Rented Bike Count','weekdays_weekend']],hue='weekdays_weekend',ax=ax1)
ax.set_title('Holiday wise monthly distribution of counts')
plt.show()

## **Feature Engineering and Correlation:**

In [ ]:
## Correlation 
plt.figure(figsize=(15,8))
correlation = rental_bike_df.corr()
sns.heatmap(abs(correlation), annot=True, cmap='coolwarm')

In [ ]:
# To remove Multicollinearity, using VIF

from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):
 
   # Calculating VIF
   vif = pd.DataFrame()
   vif["variables"] = X.columns
   vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
 
   return(vif)

In [ ]:
calc_vif(rental_bike_df[[i for i in rental_bike_df.describe().columns if i not in ['Rented Bike Count','Dew point temperature(°C)','year'] ]])

In [ ]:
# showing relation between Categorical and Dependent Features
#ploting Box plot to visualize and trying to get information from plot
for col in categorical_features:
  plt.figure(figsize=(10,8))
  sns.boxplot(x=rental_bike_df[col],y=rental_bike_df["Rented Bike Count"])
  plt.show()

In [ ]:
#treating outliers
def remove_outlier_IQR(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    rental_bike_df=df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    return rental_bike_df

In [ ]:
df_outlier_removed_SR=remove_outlier_IQR(rental_bike_df["Solar Radiation (MJ/m2)"])
df_outlier_removed_WS=remove_outlier_IQR(rental_bike_df["Wind hmmspeed (m/s)"])

df_outlier_removed_SR=pd.DataFrame(df_outlier_removed_SR)
df_outlier_removed_WS=pd.DataFrame(df_outlier_removed_WS)

ind_diff_SR=rental_bike_df.index.difference(df_outlier_removed_SR.index)
ind_diff_WS=rental_bike_df.index.difference(df_outlier_removed_WS.index)

in_first = set(ind_diff_SR)
in_second = set(ind_diff_WS)
result = in_second.union(in_first) 


for i in range(0, len(result),1):
        rental_bike_df.drop(i,inplace=True)

In [ ]:
rental_bike_df.info()

In [ ]:
rental_bike_df.Seasons.value_counts()

In [ ]:
# Create dummy variables for the catgeorical variable Seasons,Holiday,Functioning Day

rental_bike_df['Winter_Season'] = np.where(rental_bike_df['Seasons']=='Winter', 1, 0)
rental_bike_df['Spring_Season'] = np.where(rental_bike_df['Seasons']=='Spring', 1, 0)
rental_bike_df['Summer_Season'] = np.where(rental_bike_df['Seasons']=='Summer', 1, 0)
rental_bike_df['Autumn_Season'] = np.where(rental_bike_df['Seasons']=='Autumn', 1, 0)
rental_bike_df['Holiday'] = np.where(rental_bike_df['Holiday']=='No Holiday', 1, 0)
rental_bike_df['Functioning Day'] = np.where(rental_bike_df['Functioning Day']=='Yes', 1, 0)

# Drop the original column Seasons from the dataframe
rental_bike_df.drop(columns=['Seasons'],axis=1,inplace=True)


In [ ]:
rental_bike_df.head()

In [ ]:
#visualising distribution
plt.figure(figsize=(10,8))

sns.distplot(rental_bike_df['Rented Bike Count'])

In [ ]:
# square_root transformation

plt.figure(figsize=(8,6))
sns.distplot(np.sqrt(rental_bike_df['Rented Bike Count']))


In [ ]:
# drop the date column in dataset
rental_bike_df=rental_bike_df.drop(columns=['Date','day_of_week'],axis=1)


### Linear Regression

In [ ]:
#creating list of matrix to store the evaluation matrix of all model
MSE=[]
RMSE=[]
R2=[]
Adj_R2=[]


In [ ]:
# ML Model - 1 Implementation
dependent_variable = 'Rented Bike Count'

In [ ]:
# Create a list of independent variables
independent_variables = list(set(rental_bike_df.columns.tolist()) - {dependent_variable})
independent_variables

In [ ]:
# Create the data of independent variables
X = rental_bike_df[independent_variables].values

# Create the dependent variable data
y = rental_bike_df[dependent_variable].values

In [ ]:
rental_bike_df[independent_variables]

In [ ]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train[0:5]

In [ ]:
# Transforming data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train[0:5]

In [ ]:
# Fitting Multiple Linear Regression to the Training set
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
regressor.intercept_

In [ ]:
regressor.coef_

In [ ]:
y_pred_train = regressor.predict(X_train)
y_pred_train

In [ ]:
y_train

In [ ]:
# Predicting the Test set results
y_pred_test = regressor.predict(X_test)
y_pred_test

In [ ]:
y_test

In [ ]:
MSE_lr = mean_squared_error(y_test, y_pred_test)
MSE.append(MSE_lr)
MSE_lr

In [ ]:
# Train performance
math.sqrt(mean_squared_error(y_train, y_pred_train))

In [ ]:
# Test performance
RMSE_lr = math.sqrt(mean_squared_error(y_test, y_pred_test))
RMSE.append(RMSE_lr)
RMSE_lr

In [ ]:
r2_score(y_train, y_pred_train)

In [ ]:
R2_lr = r2_score(y_test, y_pred_test)
R2.append(R2_lr)
R2_lr

In [ ]:
adj_r2_lr =1-(1-r2_score(y_test,y_pred_test))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_lr)
print("Adjusted R2 : ",adj_r2_lr,'\n')

In [ ]:
  plt.figure(figsize=(10,8))
  plt.plot((y_pred_test)[:100])
  plt.plot((np.array(y_test)[:100]))
  plt.legend(["Predicted","Actual"])
  plt.show()


# **Regularised Linear Regression**

**Lasso Regression**

In [ ]:
from sklearn.linear_model import Lasso
lasso  = Lasso(alpha=0.1 , max_iter= 3000)

lasso.fit(X_train, y_train)

In [ ]:
lasso.score(X_train, y_train)

In [ ]:
y_pred_testl = lasso.predict(X_test)

In [ ]:
MSE_lasso = mean_squared_error(y_test, y_pred_testl)
MSE.append(MSE_lasso)

RMSE_lasso = math.sqrt(mean_squared_error(y_test, y_pred_testl))
RMSE.append(RMSE_lasso)

R2_lasso = r2_score(y_test, y_pred_testl)
R2.append(R2_lasso)

adj_r2_lasso =1-(1-r2_score(y_test,y_pred_testl))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_lasso)

print(f"MSE:{MSE_lasso},RMSE:{RMSE_lasso},R2:{R2_lasso},Adj_R2:{adj_r2_lasso}")

In [ ]:
  plt.figure(figsize=(10,8))
  plt.plot(y_pred_testl[:100])
  plt.plot((np.array(y_test)[:100]))
  plt.legend(["Predicted","Actual"])
  plt.show()


In [ ]:
### Cross validation
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=3)
lasso_regressor.fit(X_train, y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
y_pred_lasso = lasso_regressor.predict(X_test)

**Ridge Regression**

In [ ]:
from sklearn.linear_model import Ridge

ridge  = Ridge(alpha=0.1)

In [ ]:
ridge.fit(X_train,y_train)

In [ ]:
ridge.score(X_train, y_train)

In [ ]:
y_pred_r = ridge.predict(X_test)

In [ ]:
# Hyperprarameter tuning
ridge = Ridge()
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,10,20,30,40,45,50,55,60,100]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=3)
ridge_regressor.fit(X_train,y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,ridge_regressor.best_params_)
print("\nUsing ",ridge_regressor.best_params_, " the negative mean squared error is: ", ridge_regressor.best_score_)

In [ ]:
#Model Prediction
y_pred_ridge = ridge_regressor.predict(X_test)

In [ ]:
MSE_ridge = mean_squared_error(y_test, y_pred_ridge)
MSE.append(MSE_ridge)

RMSE_ridge = math.sqrt(mean_squared_error(y_test, y_pred_ridge))
RMSE.append(RMSE_ridge)

R2_ridge = r2_score(y_test, y_pred_ridge)
R2.append(R2_ridge)

adj_r2_ridge =1-(1-r2_score(y_test,y_pred_ridge))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_ridge)

print(f"MSE:{MSE_ridge},RMSE:{RMSE_ridge},R2:{R2_ridge},Adj_R2:{adj_r2_ridge}")

In [ ]:
  plt.figure(figsize=(10,8))
  plt.plot(y_pred_ridge[:100])
  plt.plot((np.array(y_test)[:100]))
  plt.legend(["Predicted","Actual"])
  plt.show()

**Elastic net Regression**

In [ ]:
from sklearn.linear_model import ElasticNet
#a * L1 + b * L2
#alpha = a + b and l1_ratio = a / (a + b)
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.5)

In [ ]:
elasticnet.fit(X_train,y_train)

In [ ]:
elasticnet.score(X_train, y_train)

In [ ]:
y_pred_en = elasticnet.predict(X_test)

In [ ]:
#cross validation
elastic = ElasticNet()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100],'l1_ratio':[0.3,0.4,0.5,0.6,0.7,0.8]}
elastic_regressor = GridSearchCV(elastic, parameters, scoring='neg_mean_squared_error',cv=5)
elastic_regressor.fit(X_train, y_train)

In [ ]:
y_pred_elastic = elastic_regressor.predict(X_test)

In [ ]:
print("The best fit alpha value is found out to be :" ,elastic_regressor.best_params_)
print("\nUsing ",elastic_regressor.best_params_, " the negative mean squared error is: ", elastic_regressor.best_score_)

In [ ]:
MSE_EN = mean_squared_error(y_test, y_pred_elastic)
MSE.append(MSE_EN)

RMSE_EN = math.sqrt(mean_squared_error(y_test, y_pred_elastic))
RMSE.append(RMSE_EN)

R2_EN = r2_score(y_test, y_pred_elastic)
R2.append(R2_EN)

adj_r2_EN =1-(1-r2_score(y_test,y_pred_elastic))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_EN)

print(f"MSE:{MSE_EN},RMSE:{RMSE_EN},R2:{R2_EN},Adj_R2:{adj_r2_EN}")

In [ ]:
### Heteroscedasticity
plt.scatter((y_pred_lasso),(y_test)-(y_pred_lasso))

In [ ]:
  plt.figure(figsize=(10,8))
  plt.plot(y_pred_elastic[:100])
  plt.plot((np.array(y_test)[:100]))
  plt.legend(["Predicted","Actual"])
  plt.show()

# Decision Tree

In [ ]:
rental_bike_df.shape

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
decisionTree = DecisionTreeRegressor()

param = {'max_depth' : [1,4,5,6,7]}

gridSearch_decisionTree=GridSearchCV(decisionTree,param,scoring='r2',cv=5)
gridSearch_decisionTree.fit(X_train,y_train)

decisionTree.fit(X_train,y_train)
best_DecisionTree=gridSearch_decisionTree.best_estimator_
bestDecisionTree_testScore=best_DecisionTree.score(X_test,y_test)

In [ ]:
y_pred_DT = gridSearch_decisionTree.predict(X_test)

In [ ]:
# extracting best parameters

print(f"The best Decision Tree R2 score is {gridSearch_decisionTree.best_score_} with max depth {gridSearch_decisionTree.best_params_['max_depth']}")
print('\n')
print(f"The best R2 test score is : {bestDecisionTree_testScore} with max depth = {gridSearch_decisionTree.best_params_['max_depth']}")

In [ ]:
MSE_DT = mean_squared_error(y_test, y_pred_DT)
MSE.append(MSE_DT)

RMSE_DT = math.sqrt(mean_squared_error(y_test, y_pred_DT))
RMSE.append(RMSE_DT)

R2_DT = r2_score(y_test, y_pred_DT)
R2.append(R2_DT)

adj_r2_DT =1-(1-r2_score(y_test,y_pred_DT))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_DT)

print(f"MSE:{MSE_DT},RMSE:{RMSE_DT},R2:{R2_DT},Adj_R2:{adj_r2_DT}")

In [ ]:
  plt.figure(figsize=(10,8))
  plt.plot(y_pred_DT[:100])
  plt.plot((np.array(y_test)[:100]))
  plt.legend(["Predicted","Actual"])
  plt.show()

In [ ]:
def get_features_importance (optimal_model,X_train):
  
  #shows the graph of feature importance
  
  features = X_train.columns
  importances = optimal_model.feature_importances_
  indices = np.argsort(importances)

  plt.figure(figsize=(15,10))
  plt.title('Feature Importance')
  plt.barh(range(len(indices)), importances[indices], color='red', align='center')
  plt.yticks(range(len(indices)), [features[i] for i in indices])
  plt.xlabel('Relative Importance')

  plt.show()


# Random Forest

In [ ]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
  
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(X_train,y_train)

In [ ]:
Y_train_pred_RF = regressor.predict(X_train) 

Y_test_pred_RF = regressor.predict(X_test)

In [ ]:
y_test

In [ ]:
Y_test_pred_RF

In [ ]:
MSE_RF = mean_squared_error(y_test, Y_test_pred_RF)
MSE.append(MSE_RF)

RMSE_RF = math.sqrt(mean_squared_error(y_test, Y_test_pred_RF))
RMSE.append(RMSE_RF)

R2_RF = r2_score(y_test, Y_test_pred_RF)
R2.append(R2_RF)

adj_r2_RF =1-(1-r2_score(y_test,Y_test_pred_RF))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_RF)

print(f"MSE:{MSE_RF},RMSE:{RMSE_RF},R2:{R2_RF},Adj_R2:{adj_r2_RF}")

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(Y_test_pred_RF[:100])
plt.plot((np.array(y_test)[:100]))
plt.legend(["Predicted","Actual"])
plt.show()

# XGBoost Regression

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train,y_train)

In [ ]:
y_train_pred_xgb = xgb.predict(X_train)
y_test_pred_xgb = xgb.predict(X_test)

In [ ]:
MSE_xgb = mean_squared_error(y_test, y_test_pred_xgb)
MSE.append(MSE_xgb)

RMSE_xgb = math.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
RMSE.append(RMSE_xgb)

R2_xgb = r2_score(y_test, y_test_pred_xgb)
R2.append(R2_xgb)

adj_r2_xgb =1-(1-r2_score(y_test,y_test_pred_xgb))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_xgb)

print(f"MSE:{MSE_xgb},RMSE:{RMSE_xgb},R2:{R2_xgb},Adj_R2:{adj_r2_xgb}")

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_test_pred_xgb[:100])
plt.plot((np.array(y_test)[:100]))
plt.legend(["Predicted","Actual"])
plt.show()

# LightGBM Model

In [ ]:
import lightgbm as lgb

In [ ]:
lgbr=lgb.LGBMRegressor()

In [ ]:
# finding the best parameters for XGBRegressor by gridsearchcv
lgbr_para={'n_estimators': [150,200,250],'max_depth': [7,10,13]}
lgbr_grid=GridSearchCV(estimator=lgbr,param_grid=lgbr_para,cv=5,scoring='neg_mean_squared_error',verbose=5,n_jobs=-1)

In [ ]:
lgbr_grid.fit(X_train,y_train)

In [ ]:
y_train_pred_lgbr = lgbr_grid.predict(X_train)
y_test_pred_lgbr = lgbr_grid.predict(X_test)

In [ ]:
y_train_pred_lgbr

In [ ]:
y_test_pred_lgbr

In [ ]:
MSE_lgbr = mean_squared_error(y_test, y_test_pred_lgbr)
MSE.append(MSE_lgbr)

RMSE_lgbr = math.sqrt(mean_squared_error(y_test, y_test_pred_lgbr))
RMSE.append(RMSE_lgbr)

R2_lgbr = r2_score(y_test, y_test_pred_lgbr)
R2.append(R2_lgbr)

adj_r2_lgbr =1-(1-r2_score(y_test,y_test_pred_lgbr))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
Adj_R2.append(adj_r2_lgbr)

print(f"MSE:{MSE_lgbr},RMSE:{RMSE_lgbr},R2:{R2_lgbr},Adj_R2:{adj_r2_lgbr}")

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(y_test_pred_lgbr[:100])
plt.plot((np.array(y_test)[:100]))
plt.legend(["Predicted","Actual"])
plt.show()

In [ ]:
def get_features_importance (optimal_model,X_train):
  '''
  shows the graph of feature importance
  '''
  features = pd.DataFrame(X_train).describe().columns
  importances = optimal_model.feature_importances_
  indices = np.argsort(importances)

  plt.figure(figsize=(15,10))
  plt.title('Feature Importance')
  plt.barh(range(len(indices)), importances[indices], color='green', align='center')
  plt.yticks(range(len(indices)), [independent_variables[i] for i in indices])
  plt.xlabel('Relative Importance')
  plt.show()

In [ ]:
#getting and visualizing features importance of LightGBM model
get_features_importance(lgbr_grid.best_estimator_,X_train)

# Different Model Metrices 

In [ ]:
all_model_matrices={'Mean_square_error':MSE,'Root_Mean_square_error':RMSE,'R2':R2,'Adjusted_R2':Adj_R2}
model_name=['Linear','Lasso','Ridge','Elasticnet','Decision_Tree','Random_Forest','xgboost','LightGBM']

matrices_df=pd.DataFrame.from_dict(all_model_matrices,orient="index",columns=model_name)

In [ ]:
matrices_df

# Model Explainability

## Shap

In [ ]:
!pip install shap

In [ ]:
from sklearn.tree import export_graphviz
import shap 
sns.set_style('darkgrid')

In [ ]:
for_instance=2

In [ ]:
#shap for lightGBM model
# Initialize JavaScript visualizations in notebook environment
shap.initjs()
# Define a tree explainer for the built model
explainer = shap.TreeExplainer(lgbr_grid.best_estimator_)
# obtain shap values for the first row of the test data
shap_values = explainer(X_test,check_additivity=False)
# # visualize the first prediction's explanation with a force plot
shap.plots.force(shap_values[for_instance])

## ELI 5

In [ ]:
pip install eli5

In [ ]:
import eli5 as eli

In [ ]:
for_instance=2

In [ ]:
eli.show_prediction(lgbr_grid.best_estimator_, pd.DataFrame(X_test).reset_index(drop=True).loc[[for_instance]],
                    feature_names=independent_variables[:],
                    show_feature_values=True)



1.   The green color shows how much the feature contributes to the prediction of the respective class and the weights are positive for the green color.

2. The red color has negative weights that indicate the feature isn't contributing to the prediction of that class.

3. It can be observed from the above output, eli5 shows us the contribution of each feature in predicting the output.


# **Conclusion**

## Data Exploration Conclusions

In this project, we explored several types of informations that influence bike rental count. Below is a quick summary of exploratory data analysis
conclusions from above pie chart:

 **Season:** Most bikes have been rented in the summer season.least bike rent count is in winter season.autumn and spring seasons have almost equal amounts of bike rent count.

 **YearwiseRentedBikeCount:** Most of the bikes have been rented in the year 2018.

**MonthwiseRentedBikeCount in year 2018:**Very less bikes have been rented in december which is winter season.

**Intensities of Rainfall:**People tend to rent bikes when there is no or less rainfall.

**Intensities of Snowfall:**People tend to rent bikes when there is no or less snowfall.

**Temperature:** People generally prefer to bike at moderate to high temperatures. We see highest rental counts between 25 to 36 degree celcius.

**Visibility Range:** people tend to rent bikes when the visibility is between 300 to 1700.

**Working or Non-working Day:** We see 2 rental patterns across the day in bike rentals count - first for a Working Day where the rental count high at peak office hours (8am and 5pm) and the second for a Non-working day where rental count is more or less uniform across the day with a peak at around noon.

**Hour of the day:** Bike rental count is mostly correlated with the time of the day. As indicated above, the count reaches a high point during peak hours on a working day and is mostly uniform during the day on a non-working day.


#Modelling Conclusions:

We used 6 Regression Models to predict the bike rental count at any hour of the day - Linear Regression, Ridge, Lasso, Random Forest,XG Boost and LightGBM Model.

**Below is a summary of the model performances:**

Of all the models, we found a LightGBM Model providing the best/lowest RMSE score and highest R^2 score.
Hour of the day is the most important feature in the respect of all independent feature which provide highest bike rented count.
Thus, we have successfully built predictive models that can predict the demand for rental bikes based on different weather conditions and all other features. 
 If the model interpretability is important to the stakeholders, we can choose deploy the Light GBM model.


